<a href="https://colab.research.google.com/github/Vakhranev/Gramota/blob/main/%D0%A2%D0%B5%D0%B3%D0%B8%20%D0%BB%D0%B8%D0%BD%D0%B3%D0%B2%D0%B8%D1%81%D1%82%D0%BE%D0%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import glob

def read_excel_file(file):
    # Чтение файла и игнорирование возможных заголовков
    df = pd.read_excel(file, header=None)

    # Проверка наличия заголовка
    if pd.api.types.is_numeric_dtype(df.iloc[0, 0]) and pd.api.types.is_numeric_dtype(df.iloc[1, 0]):
        # Если первая и вторая строки числовые, то заголовка нет, и устанавливаем собственный заголовок
        df.columns = range(df.shape[1])
    else:
        # Если заголовок есть, читаем файл снова, чтобы корректно задать header
        df = pd.read_excel(file)

    return df

# Считывание всех файлов .xls и .xlsx из текущей директории
all_files = glob.glob("*.xls*")

# Инициализация пустого DataFrame для хранения всех данных
combined_df = pd.DataFrame()

# Обработка каждого файла по отдельности
for file in all_files:
    df = read_excel_file(file)

    # Поиск столбца с нумерацией
    start_col_index = None
    for i, col in enumerate(df.columns):
        if pd.api.types.is_numeric_dtype(df.iloc[:, i]):
            start_col_index = i
            break

    if start_col_index is not None:
        # Выбор только 4 столбцов начиная со столбца с нумерацией
        df = df.iloc[:, start_col_index:start_col_index+5]

        # Удаление строк, где в пятом столбце есть цифры (если такой столбец существует)
        if df.shape[1] == 5:
            df = df[~df.iloc[:, 4].apply(lambda x: isinstance(x, (int, float)))]

        # Удаление пустых строк и строк, где не все ячейки заполнены (кроме первого столбца)
        df = df.dropna(how='any').reset_index(drop=True)

        # Объединение с главным DataFrame
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# Убедимся, что у нас есть только пять столбцов в итоговом DataFrame
combined_df = combined_df.iloc[:, :5]

# Обновление нумерации в первом столбце
combined_df.iloc[:, 0] = range(1, len(combined_df) + 1)

# Удаление дубликатов строк, кроме первой (нумерации)
combined_df = combined_df.drop_duplicates(subset=combined_df.columns[1:])

# Обновление нумерации после удаления дубликатов
combined_df.iloc[:, 0] = range(1, len(combined_df) + 1)

# Сохранение объединенного DataFrame в новый Excel файл
combined_df.columns = ['Нумерация', 'Столбец 1', 'Столбец 2', 'Столбец 3', 'Столбец 4']
combined_df.to_excel("combined.xlsx", index=False)

print("Объединение завершено и сохранено в 'combined.xlsx'")

Объединение завершено и сохранено в 'combined.xlsx'


In [4]:
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 4.0 MB/s eta 0:00:00


In [11]:
from docx import Document
import re

# Открываем документ
doc = Document('Матрица общая_ИТОГ.docx')

# Функция для извлечения тегов из таблиц
def extract_tags_from_tables(doc):
    tags = set()
    for table in doc.tables:
        for row in table.rows:
            for cell in row.cells:
                cell_text = cell.text.strip()
                if cell_text:
                    cell_tags = cell_text.split('\n')
                    tags.update(cell_tags)
    return tags

# Функция для извлечения тегов из текста после указанной фразы
def extract_additional_tags(doc, start_phrase):
    tags = set()
    start_found = False
    for para in doc.paragraphs:
        if start_found:
            para_text = para.text.strip()
            if para_text.startswith('- '):
                tag = para_text[2:].strip()
                tags.add(tag)
        elif start_phrase in para.text:
            start_found = True
    return tags

# Извлечение тегов из таблиц и текста после указанной фразы
table_tags = extract_tags_from_tables(doc)
additional_tags = extract_additional_tags(doc, "Дополнительные термины, которые могут быть добавлены в случае необходимости в маркировку вопроса любого из разделов:")

# Объединяем все теги
all_tags = table_tags.union(additional_tags)

# Очистка тегов от знаков пунктуации и приведение к нижнему регистру
cleaned_tags = {re.sub(r'[^\w\s]', '', tag).lower() for tag in all_tags}

In [8]:
cleaned_tags

{'',
 'а или о',
 'а или я',
 'аббревиатура',
 'аббревиация',
 'авторский знак',
 'антонимы',
 'апостроф',
 'архаизм',
 'афоризм',
 'беглая',
 'безличный',
 'безударная',
 'беспредложное',
 'бессоюзное',
 'бессуффиксный',
 'библиография',
 'благозвучие',
 'бранное',
 'будущее',
 'буква й',
 'буква ё',
 'в вузе',
 'в школе ',
 'варваризм',
 'вариант',
 'варианты',
 'вводное',
 'вводное слово',
 'вид',
 'возвратное',
 'возвратный',
 'вопросительное',
 'вопросительный',
 'восклицательный',
 'время',
 'время ',
 'вставная конструкция',
 'вставные',
 'выбор предлога',
 'выбор слова ',
 'выделение',
 'главное',
 'глагол',
 'гласная буква согласная буква',
 'гласный звук',
 'градус',
 'графика',
 'дательный винительный',
 'двоеточие ',
 'двойные согласные',
 'деепричастие',
 'деепричастный оборот',
 'действительный ',
 'деловое',
 'деловой',
 'деловые бумаги',
 'дефис',
 'диалект',
 'диалектизм',
 'диалог',
 'для иностранцев',
 'долгий',
 'дополнение',
 'дробное',
 'е или и',
 'е или э',
 'ед

In [12]:
import pandas as pd

# Загрузка таблицы
df = pd.read_excel('combined (5).xlsx')

# Функция для замены " / " на "; "
def replace_slash(text):
    if isinstance(text, str):
        return text.replace(' / ', '; ')
    return text

# Функция для удаления начальных "; "
def remove_initial_semicolon(text):
    if isinstance(text, str):
        return text.lstrip('; ').strip()
    return text

# Функция для разделения тегов, удаления дубликатов и оставления только одного экземпляра каждого тега
def separate_tags(text, tag_set):
    if isinstance(text, str):
        # Очистка текста от пунктуации и приведение к нижнему регистру
        cleaned_text = re.sub(r'[^\w\s]', '', text).lower()
        # Поиск тегов
        matched_tags = [tag for tag in tag_set if tag in cleaned_text]
        # Удаление дубликатов
        unique_tags = list(set(matched_tags))
        return '; '.join(unique_tags)
    return text

# Применяем функции к столбцу E
df['Столбец 4'] = df['Столбец 4'].apply(replace_slash)
df['Столбец 4'] = df['Столбец 4'].apply(remove_initial_semicolon)
df['Столбец 4'] = df['Столбец 4'].apply(lambda x: x if ' / ' in str(x) else separate_tags(x, cleaned_tags))

# Удаление строк с пустыми значениями в столбце E
df = df.dropna(subset=['Столбец 4']).reset_index(drop=True)

# Поиск и вывод тегов, которые ни разу не были использованы
used_tags = set()
for tags in df['Столбец 4']:
    if isinstance(tags, str):
        used_tags.update(tags.split('; '))

unused_tags = cleaned_tags - used_tags
print("Unused tags:", unused_tags)

# Сохранение измененной таблицы
df.to_excel('combined_updated.xlsx', index=False)

Unused tags: {'бранное', 'отрицательное', 'степень сравнительная превосходная', 'невозвратный', 'сращение', 'жаргонное', 'плюс', 'у или ю', 'ю или я', 'шрифт', 'сложносокращенное одушевленное', 'после ц', 'переходный непереходный', 'звательный', 'несовершенный двувидовой', 'в вузе', 'уточняющие пояснительные', 'интернет', 'в школе ', 'нулевой', 'родственные слова', 'условное', 'научное', 'трансплантация', 'нетематический', 'дательный винительный', 'минус', 'разноспрягаемый', 'непроверяемая чередующаяся', 'твердый знак', 'постфикс', 'опечатка', 'долгий', 'номер', 'авторский знак', 'косвенная речь', 'недолгий', 'способ действия', 'унификс', 'суффиксоид', 'повторяющиеся слова ', 'калька', 'синхронический диахронический', 'побочное', 'религиозная ', 'указательное', 'эргоним ', 'а или я', 'книжный', 'редукция', 'гласная буква согласная буква', 'переход из одной части речи в другую', 'крылатое выражение', 'цифра', 'небуквенный знак', 'возвратное', 'градус', 'период', 'префиксоид'}


In [13]:
import pandas as pd

# Загрузка таблицы
df = pd.read_excel('combined_updated.xlsx')

# Функция для удаления начальных "; "
def remove_initial_semicolon(text):
    if isinstance(text, str):
        return text.lstrip('; ').strip()
    return text

# Функция для удаления пробелов перед ";"
def remove_spaces_before_semicolon(text):
    if isinstance(text, str):
        return re.sub(r'\s+;', ';', text)
    return text

# Применяем функции к столбцу E
# Удаление строк с пустыми значениями в столбце E
df = df.dropna(subset=['Столбец 4']).reset_index(drop=True)

# Удаление начальных "; "
df['Столбец 4'] = df['Столбец 4'].apply(remove_initial_semicolon)

# Удаление пробелов перед ";"
df['Столбец 4'] = df['Столбец 4'].apply(remove_spaces_before_semicolon)

# Сохранение измененной таблицы
df.to_excel('combined_updated_cleaned.xlsx', index=False)